In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

from pyspark.sql import *

from pyspark.sql import Window
from pyspark.sql.functions import rank, col


import pyspark.sql.functions as F 

In [7]:
#Creacion de dataset
df = (spark
  .read                                              
  .option("inferSchema","true")                 
  .option("header","true")                           
  .csv("rating.csv"))

## feature enginering

In [8]:
def nb_previous_ratings(df):
  df=df.withColumn("nb_previous_ratings",rank().over(Window.partitionBy("userId").orderBy("timestamp")) -1 )
  return df

def avg_ratings_previous(df):
  WindowSpec = Window.partitionBy("userId").rowsBetween(Window.unboundedPreceding,-1)

  df = df.withColumn("avg_ratings_previous", F.avg(F.col("rating")).over(WindowSpec))
  df = df.orderBy("userId", "timestamp")
  return df

In [9]:
df = nb_previous_ratings(df)
df.show()

+------+-------+------+-------------------+-------------------+
|userId|movieId|rating|          timestamp|nb_previous_ratings|
+------+-------+------+-------------------+-------------------+
|   148|     86|   2.0|2002-04-16 14:06:04|                  0|
|   148|    908|   4.0|2002-04-16 14:06:04|                  0|
|   148|   2103|   2.0|2002-04-16 14:06:04|                  0|
|   148|   1210|   5.0|2002-04-16 14:06:26|                  3|
|   148|   1097|   5.0|2002-04-16 14:06:56|                  4|
|   148|   1453|   3.0|2002-04-16 14:06:56|                  4|
|   148|   1968|   3.0|2002-04-16 14:06:56|                  4|
|   148|     18|   1.0|2002-04-16 14:07:27|                  7|
|   148|    368|   3.0|2002-04-16 14:07:27|                  7|
|   148|   1270|   5.0|2002-04-16 14:07:27|                  7|
|   148|   5218|   4.0|2002-04-16 14:08:44|                 10|
|   148|   5009|   2.0|2002-04-16 14:09:29|                 11|
|   148|   5067|   3.0|2002-04-16 14:10:

In [10]:
df = avg_ratings_previous(df)
df.show()

+------+-------+------+-------------------+-------------------+--------------------+
|userId|movieId|rating|          timestamp|nb_previous_ratings|avg_ratings_previous|
+------+-------+------+-------------------+-------------------+--------------------+
|     1|    924|   3.5|2004-09-10 03:06:38|                  0|                null|
|     1|    919|   3.5|2004-09-10 03:07:01|                  1|                 3.5|
|     1|   2683|   3.5|2004-09-10 03:07:30|                  2|                 3.5|
|     1|   1584|   3.5|2004-09-10 03:07:36|                  3|                 3.5|
|     1|   1079|   4.0|2004-09-10 03:07:45|                  4|                 3.5|
|     1|    653|   3.0|2004-09-10 03:08:11|                  5|                 3.6|
|     1|   2959|   4.0|2004-09-10 03:08:18|                  6|                 3.5|
|     1|    337|   3.5|2004-09-10 03:08:29|                  7|  3.5714285714285716|
|     1|   1304|   3.0|2004-09-10 03:08:40|                  8|  

In [10]:
df.repartition(1).write.format('com.databricks.spark.csv').save("./data/",header = 'true')
print("Process finish")